In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset/data.csv")

In [2]:
!lsb_release -a

'lsb_release' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [3]:
!nvidia-smi

Fri Feb 14 16:28:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| 30%   28C    P8             N/A /   75W |     605MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!nvcc -V

'nvcc' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [ ]:
# 切換目錄 (Colab 預設目錄為 /content，使用 %cd 切換目錄)


# 微調模型

In [5]:
# 安裝套件
!pip install torch torchvision torchaudio transformers datasets evaluate accelerate scikit-learn

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   --------------------------------- ------ 1.3/1.6 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 23.3 MB/s eta 0:00:00


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

AttributeError: module 'torch' has no attribute 'is_available'

In [ ]:
'''
AutoTokenizer：這有助於將我們的文字資料標記為 BERT 可以理解的格式。 「Auto」前綴意味著它可以為各種模型推斷適當的分詞器。
AutoModelForSequenceClassification：一個通用的類別，是用於「序列分類」任務的模型架構。「Auto」前綴使其在各種預訓練模型中具有通用性。
TrainingArguments：定義訓練配置的設定，例如 learning rateb、batch size 和 epoch。
Trainer：用於訓練和評估，使 finetune 變得簡單。
pipeline：使用模型的模型。
DataCollat​​eWithPadding：確保我們分詞化後的資料，以一致的長度串接在一起，並在必要時增加 padding。這對於訓練的穩定性和效率至關重要。
'''
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

import random
from sklearn.metrics import f1_score


'''
函式
'''
# 讀取 .txt 文件
def load_dataset_from_file(file_path, seed=42):
    # 讀檔
    with open(file_path, "r", encoding='utf-8') as file:
        # 將每一行資料以 list 型態回傳
        lines = file.readlines()

        # 洗牌 (記得設定 random seed，確保每次洗牌結果一樣)
        random.seed(seed)
        random.shuffle(lines)

        # 整合訓練資料
        sentences = []
        labels = []

        # 逐行讀取資料
        for line in lines:
            # 每一行資料的 tab (\t) 作為分隔符號
            parts = line.strip().split('\t')

            # 確保每一行資料都有兩個部分
            if len(parts) == 2:
                sentences.append(parts[0])
                labels.append(int(parts[1]))
            else:
                print(f'格式錯誤的行號: {line}')
        return sentences, labels
    
# 轉換成 huggingface trainer 可以使用的 datasets
def convert_to_dataset(sentences, labels, tokenizer, max_seq_length):
    # 建立 Dataset
    dataset = Dataset.from_dict({
        'sentences': sentences,
        'labels': labels
    })

    # 回傳切分資料 (訓練 和 驗證)
    dataset = dataset.train_test_split(test_size=0.2)
    '''
    print(dataset) 的內容如下:

    DatasetDict({
        train: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 6212
        })
        test: Dataset({
            features: ['sentences', 'labels'],
            num_rows: 1554
        })
    })
    '''

    # 預處理資料
    def preprocess_data(dataset):
        # 將句子轉換為 token (tokenization)
        return tokenizer(
            dataset['sentences'], 
            truncation=True, 
            padding=True, 
            return_tensors='pt', 
            max_length=max_seq_length
        )

    # 轉換資料
    train_data = dataset['train'].map(preprocess_data, batched=True)
    valid_data = dataset['test'].map(preprocess_data, batched=True)

    return DatasetDict({
        'train': train_data,
        'test': valid_data
    })

# 計算模型評估指標
def compute_metrics(predicted_results):
    labels = predicted_results.label_ids
    preds = predicted_results.predictions.argmax(-1)
    '''
    為什麼是 argmax(-1)？

    import numpy as np
    predictions = np.array([
        [0.1, 0.9],
        [0.8, 0.2],
        [0.4, 0.6],
    ])
    preds = predictions.argmax(axis=-1)
    print(preds) 

    np.argmax() 會找出最大的值，並回傳索引值
    - 如果是 axis=0，則會是 [1, 0]，因為由上而下，對每一個欄位取最大值
    - 如果是 axis=1，則會是 [1, 0, 1]，因為由左而右，對每一個列取最大值
    - 如果是 axis=-1，則會以最後一個維度為基礎，取最大值
    '''

    f1 = f1_score(labels, preds, average='binary') # binary, micro, macro, weighted
    return {
        'f1': f1,
    }